In [4]:
# install neccesary plugins
!pip install pandas
!pip install selenium
!pip install webdriver_manager
!pip install bs4

You should consider upgrading via the 'c:\users\mn850\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mn850\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\mn850\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from datetime import datetime
from webdriver_manager.firefox import GeckoDriverManager
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Driver [C:\Users\mn850\.wdm\drivers\geckodriver\win64\v0.29.1\geckodriver.exe] found in cache


In [23]:
defi_dict = {}

In [40]:
driver.get('https://defillama.com/protocols/staking')
category = 'Staking'
xpath_pagenumber = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[2]'
number_of_pages = int(driver.find_element_by_xpath(xpath_pagenumber).text.split()[-1:][0])
number_of_pages

1

In [41]:
protocol = 'sc-caSCKo hKzJw'
token = 'sc-caSCKo dIpawx'
platform = 'sc-ksYbfQ jqDzNU css-4cffwv'
tvl = 'sc-ksYbfQ jqDzNU css-4cffwv'
xpath_next_page = '/html/body/div/div/div/div[2]/div/div/div[2]/div/div[4]/div[3]/div'




for page in range(1,number_of_pages +1):
    print(page)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    html = soup.find_all("div", class_="sc-cHGsZl dBFEZZ css-1yh09yi")
    rows = html[0].find_all('div', attrs={'style': 'height: 48px;'})
    

    
    for rownumer in range(len(rows)):
        defi_dict[rows[rownumer].find_all('div', attrs={'class': protocol})[0].text] = \
             [category,
              rows[rownumer].find_all('div', attrs={'class': token})[0].text, 
              rows[rownumer].find_all('div', attrs={'class': platform})[0].text,
              rows[rownumer].find_all('div')[11].text,
              rows[rownumer].find_all('div')[13].text,
              rows[rownumer].find_all('div')[9].text,
              rows[rownumer].find_all('div', attrs={'class': tvl})[2].text]

    driver.find_element_by_xpath(xpath_next_page).click()
    


1


In [75]:
df=pd.DataFrame.from_dict(defi_dict,orient='index',
                          columns = ['Category','Symbol', 'Chain', '1h Change', '1d Change', '7d Change', 'TVL'])
df['TVL_converted'] = df['TVL'].str.replace('$','', regex = True).replace(',','', regex = True)
df['Snapshot_taken'] = datetime.now()
df = df.reset_index().rename(columns = {'index':'Protocol'})
df

,Protocol,Category,Symbol,Chain,1h Change,1d Change,7d Change,TVL,TVL_converted,Snapshot_taken
0,AAVE,Lending,AAVE,Ethereum,+8.59%,+64.58%,-0.08%,$9.93b,9.93b,2021-04-30 23:59:00.921556
1,PancakeSwap,Dexes,CAKE,Binance,+2.32%,+13.12%,-0.11%,$9.93b,9.93b,2021-04-30 23:59:00.921556
2,MakerDAO,Protocols,MKR,Ethereum,+7.88%,+9.54%,+0.08%,$9.75b,9.75b,2021-04-30 23:59:00.921556
3,WBTC,Assets,WBTC,Ethereum,+7.34%,+15.76%,-0.13%,$9.2b,9.2b,2021-04-30 23:59:00.921556
4,Compound,Lending,COMP,Ethereum,+2.28%,+3.96%,-0.03%,$9b,9b,2021-04-30 23:59:00.921556
...,...,...,...,...,...,...,...,...,...,...
183,Shell Protocol,Dexes,-,Ethereum,-2.20%,"$114,679",0%,"$114,679",114679,2021-04-30 23:59:00.921556
184,SynLev,Protocols,SYN,Ethereum,+1.25%,+17.04%,-0.55%,"$100,676",100676,2021-04-30 23:59:00.921556
185,Augur,Dexes,REP,Ethereum,0%,"$75,013",0%,"$75,013",75013,2021-04-30 23:59:00.921556
186,Robo-Advisor fo...,Yield,-,Ethereum,+0.58%,+4.84%,+0.01%,"$45,836",45836,2021-04-30 23:59:00.921556


In [76]:
def value_to_float(x):
    if 'b' in x:
        return float(x.replace('b', '')) * 1000000000
    if 'm' in x:
        return float(x.replace('m', '')) * 1000000
    else:
        return x

df['TVL_converted'] = df['TVL_converted'].apply(value_to_float)
df['TVL_converted']

0      9930000000.0
1      9930000000.0
2      9750000000.0
3      9200000000.0
4      9000000000.0
           ...     
183          114679
184          100676
185           75013
186           45836
187            4172
Name: TVL_converted, Length: 188, dtype: object

In [77]:
df.to_csv('Defi llama stats ' + datetime.now().strftime('%Y.%m.%d - %Hh%Mmin') + '.csv',
          index = False)